# [1327. List the Products Ordered in a Period](https://leetcode.com/problems/list-the-products-ordered-in-a-period/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Products

<pre>+------------------+---------+
| Column Name      | Type    |
+------------------+---------+
| product_id       | int     |
| product_name     | varchar |
| product_category | varchar |
+------------------+---------+</pre>
product_id is the primary key (column with unique values) for this table.
This table contains data about the company's products.


Table: Orders

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| order_date    | date    |
| unit          | int     |
+---------------+---------+</pre>
This table may have duplicate rows.
product_id is a foreign key (reference column) to the Products table.
unit is the number of products ordered in order_date.


Write a solution to get the names of products that have at least 100 units ordered in February 2020 and their amount.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Products table:
<pre>+-------------+-----------------------+------------------+
| product_id  | product_name          | product_category |
+-------------+-----------------------+------------------+
| 1           | Leetcode Solutions    | Book             |
| 2           | Jewels of Stringology | Book             |
| 3           | HP                    | Laptop           |
| 4           | Lenovo                | Laptop           |
| 5           | Leetcode Kit          | T-shirt          |
+-------------+-----------------------+------------------+</pre>
Orders table:
<pre>+--------------+--------------+----------+
| product_id   | order_date   | unit     |
+--------------+--------------+----------+
| 1            | 2020-02-05   | 60       |
| 1            | 2020-02-10   | 70       |
| 2            | 2020-01-18   | 30       |
| 2            | 2020-02-11   | 80       |
| 3            | 2020-02-17   | 2        |
| 3            | 2020-02-24   | 3        |
| 4            | 2020-03-01   | 20       |
| 4            | 2020-03-04   | 30       |
| 4            | 2020-03-04   | 60       |
| 5            | 2020-02-25   | 50       |
| 5            | 2020-02-27   | 50       |
| 5            | 2020-03-01   | 50       |
+--------------+--------------+----------+</pre>
Output:
<pre>+--------------------+---------+
| product_name       | unit    |
+--------------------+---------+
| Leetcode Solutions | 130     |
| Leetcode Kit       | 100     |
+--------------------+---------+</pre>
Explanation:
Products with product_id = 1 is ordered in February a total of (60 + 70) = 130.
Products with product_id = 2 is ordered in February a total of 80.
Products with product_id = 3 is ordered in February a total of (2 + 3) = 5.
Products with product_id = 4 was not ordered in February 2020.
Products with product_id = 5 is ordered in February a total of (50 + 50) = 100.

In [2]:
# pandas schema

import pandas as pd

data = [[1, 'Leetcode Solutions', 'Book'], [2, 'Jewels of Stringology', 'Book'], [3, 'HP', 'Laptop'],
        [4, 'Lenovo', 'Laptop'], [5, 'Leetcode Kit', 'T-shirt']]
products = pd.DataFrame(data, columns=['product_id', 'product_name', 'product_category']).astype(
    {'product_id': 'Int64', 'product_name': 'object', 'product_category': 'object'})
data = [[1, '2020-02-05', 60], [1, '2020-02-10', 70], [2, '2020-01-18', 30], [2, '2020-02-11', 80],
        [3, '2020-02-17', 2], [3, '2020-02-24', 3], [4, '2020-03-01', 20], [4, '2020-03-04', 30], [4, '2020-03-04', 60],
        [5, '2020-02-25', 50], [5, '2020-02-27', 50], [5, '2020-03-01', 50]]
orders = pd.DataFrame(data, columns=['product_id', 'order_date', 'unit']).astype(
    {'product_id': 'Int64', 'order_date': 'datetime64[ns]', 'unit': 'Int64'})

from pyspark.sql.functions import *
from pyspark.sql import SparkSession

#pyspark schema

spark = SparkSession.builder.getOrCreate()

products_df = spark.createDataFrame(products)
orders_df = spark.createDataFrame(orders)

In [3]:
products_df.show(truncate=False)

+----------+---------------------+----------------+
|product_id|product_name         |product_category|
+----------+---------------------+----------------+
|1         |Leetcode Solutions   |Book            |
|2         |Jewels of Stringology|Book            |
|3         |HP                   |Laptop          |
|4         |Lenovo               |Laptop          |
|5         |Leetcode Kit         |T-shirt         |
+----------+---------------------+----------------+



In [4]:
orders_df.show(truncate=False)

+----------+-------------------+----+
|product_id|order_date         |unit|
+----------+-------------------+----+
|1         |2020-02-05 00:00:00|60  |
|1         |2020-02-10 00:00:00|70  |
|2         |2020-01-18 00:00:00|30  |
|2         |2020-02-11 00:00:00|80  |
|3         |2020-02-17 00:00:00|2   |
|3         |2020-02-24 00:00:00|3   |
|4         |2020-03-01 00:00:00|20  |
|4         |2020-03-04 00:00:00|30  |
|4         |2020-03-04 00:00:00|60  |
|5         |2020-02-25 00:00:00|50  |
|5         |2020-02-27 00:00:00|50  |
|5         |2020-03-01 00:00:00|50  |
+----------+-------------------+----+



In [15]:
# solving in spark dataframe
orders_df.alias('o')\
         .join(products_df.alias('p'),['product_id'],'inner')\
         .where(col('order_date').like('2020-02%'))\
         .groupBy('product_id','product_name').agg(sum('unit').alias('unit'))\
         .where('unit>=100')\
         .show()

+----------+------------------+----+
|product_id|      product_name|unit|
+----------+------------------+----+
|         1|Leetcode Solutions| 130|
|         5|      Leetcode Kit| 100|
+----------+------------------+----+



In [7]:
# Solving in Spark SQL

orders_df.createOrReplaceTempView('orders')
products_df.createOrReplaceTempView('products')

spark.sql('''
select product_name,sum(unit) as unit
from products join orders using (product_id)
where order_date like '2020-02%'
group by product_id,product_name
having sum(unit) >= 100
''').show()

+------------------+----+
|      product_name|unit|
+------------------+----+
|Leetcode Solutions| 130|
|      Leetcode Kit| 100|
+------------------+----+



In [16]:
spark.stop()